# Import Modules

In [ ]:
# Import modules
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from torch.nn import init
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Preprocessing & Statistical Analysis

In [ ]:
# Load the dataset from CSV file
data = pd.read_csv('Crop_recommendation.csv')

In [ ]:
# Features and their explanations
features = ['N - Nitrogen content', 'P - Phosphorous content', 'K - Potassium content', 'Temperature', 'Humidity', 'pH', 'Rainfall']
explanations = [
    'Ratio of Nitrogen content in soil',
    'Ratio of Phosphorous content in soil',
    'Ratio of Potassium content in soil',
    'Temperature in degree Celsius',
    'Relative humidity in %',
    'pH value of the soil',
    'Rainfall in mm'
]

# Create a table trace
table_trace = go.Table(
    header=dict(values=['Feature', 'Explanation'],
                fill=dict(color='#6495ED'),
                font=dict(color='white', size=12),
                align=['center', 'center']),
    cells=dict(values=[features, explanations],
               fill=dict(color='#F0F8FF'),
               font=dict(color='black', size=12),
               align=['left', 'left']))  # Align text to the left

# Create layout
layout = go.Layout(
    title='Feature Explanations',
    margin=dict(l=20, r=20, t=30, b=10),
    height=250,
    width=700,
    title_x=0.5,
    title_y=0.95,
)

# Create figure
fig = go.Figure(data=[table_trace], layout=layout)

# Show the figure
fig.show()

In [ ]:
# Check for missing values
print(data.isnull().sum())

In [ ]:
# Select columns 1 to 7
selected_columns = data.iloc[:, 0:7]

# Convert selected columns to numeric data types
selected_columns = selected_columns.apply(pd.to_numeric, errors='coerce')

# Calculate quartiles and the IQR
Q1 = selected_columns.quantile(0.25)
Q3 = selected_columns.quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = (selected_columns < lower_bound) | (selected_columns > upper_bound)

# Check if there are any outliers
if outliers.any().any():
    print("There are outliers in the dataset.")
else:
    print("There are no outliers in the dataset.")

In [ ]:
# Calculate summary statistics and round to two decimal places
summary_stats = data.describe().round(2).reset_index()

# Create the Plotly figure
fig = go.Figure(data=[go.Table(
    header=dict(values=summary_stats.columns,
                fill_color='#6495ED',  # Set header fill color
                font=dict(color='white', size=12),  # Set header font
                align='center'),  # Set header alignment
    cells=dict(values=[summary_stats[col] for col in summary_stats.columns],
               fill_color='#F0F8FF',  # Set cell fill color
               font=dict(color='black', size=12),  # Set cell font
               align='center'),  # Set cell alignment
)])

# Update table layout
fig.update_layout(
    title='Summary Statistics',
    margin=dict(l=20, r=20, t=30, b=10),
    height=250,
    width=700,
    title_x=0.5,
    title_y=0.95,
)

# Show table
fig.show()

In [ ]:
# Histograms
plt.figure(figsize=(12, 7))
plt.subplot(2, 4, 1)
sns.histplot(data['N'], bins=20, kde=True, color='skyblue')
plt.title('Distribution of Nitrogen')

plt.subplot(2, 4, 2)
sns.histplot(data['P'], bins=20, kde=True, color='salmon')
plt.title('Distribution of Phosphorous')

plt.subplot(2, 4, 3)
sns.histplot(data['K'], bins=20, kde=True, color='green')
plt.title('Distribution of Potassium')

plt.subplot(2, 4, 4)
sns.histplot(data['temperature'], bins=20, kde=True, color='orange')
plt.title('Distribution of Temperature')

plt.subplot(2, 4, 5)
sns.histplot(data['humidity'], bins=20, kde=True, color='purple')
plt.title('Distribution of Humidity')

plt.subplot(2, 4, 6)
sns.histplot(data['ph'], bins=20, kde=True, color='brown')
plt.title('Distribution of pH')

plt.subplot(2, 4, 7)
sns.histplot(data['rainfall'], bins=20, kde=True, color='blue')
plt.title('Distribution of Rainfall')

plt.tight_layout()
plt.show()

In [ ]:
# Box plots
plt.figure(figsize=(12, 7))
plt.subplot(2, 4, 1)
sns.boxplot(y=data['N'], color='skyblue')
plt.title('Box plot of Nitrogen')

plt.subplot(2, 4, 2)
sns.boxplot(y=data['P'], color='salmon')
plt.title('Box plot of Phosphorous')

plt.subplot(2, 4, 3)
sns.boxplot(y=data['K'], color='green')
plt.title('Box plot of Potassium')

plt.subplot(2, 4, 4)
sns.boxplot(y=data['temperature'], color='orange')
plt.title('Box plot of Temperature')

plt.subplot(2, 4, 5)
sns.boxplot(y=data['humidity'], color='purple')
plt.title('Box plot of Humidity')

plt.subplot(2, 4, 6)
sns.boxplot(y=data['ph'], color='brown')
plt.title('Box plot of pH')

plt.subplot(2, 4, 7)
sns.boxplot(y=data['rainfall'], color='blue')
plt.title('Box plot of Rainfall')

plt.tight_layout()
plt.show()

In [ ]:
# Create a DataFrame from the data
df = pd.DataFrame(selected_columns)

# Calculate the correlation matrix
corr_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap between Features')
plt.show()

In [ ]:
# This function generates a summary plot of crop data, displaying the top and bottom crops based on a specified feature,
# with the corresponding values, using horizontal bar charts in a subplot layout.
def plot_crop_summary(data, feature, color_top, color_last, title, feature_full_name):
    # Calculate crop summary
    crop_summary = pd.pivot_table(data, index='label', aggfunc='mean')
    crop_summary_feature = crop_summary.sort_values(by=feature, ascending=False)
    
    # Round the feature values to two decimal places for consistency
    crop_summary_feature[feature] = crop_summary_feature[feature].round(2)
    
    # Create subplot
    fig = make_subplots(rows=1, cols=2)

    top = {
        'y': crop_summary_feature[feature][0:10].sort_values().index,
        'x': crop_summary_feature[feature][0:10].sort_values()
    }

    last = {
        'y': crop_summary_feature[feature][-10:].index,
        'x': crop_summary_feature[feature][-10:]
    }

    # Add bar charts for top and bottom crops
    fig.add_trace(
        go.Bar(y=top['y'], x=top['x'], name="Most " + feature_full_name + " required",
               marker_color=color_top, orientation='h', text=top['x'], hoverinfo='x+text',
               hovertemplate='%{x:.2f}'),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(y=last['y'], x=last['x'], name="Least " + feature_full_name + " required",
               marker_color=color_last, orientation='h', text=last['x'], hoverinfo='x+text',
               hovertemplate='%{x:.2f}'),
        row=1, col=2
    )

    # Update trace and layout
    fig.update_traces(texttemplate='%{text}', textposition='inside')
    fig.update_layout(
        title_text=title, title_x=0.5, plot_bgcolor='white', font_size=12, font_color='black', height=500
    )

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    
    # Show plot
    fig.show()

In [ ]:
# Plots
plot_crop_summary(data, 'N', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of Nitrogen", "nitrogen")
plot_crop_summary(data, 'P', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of Phosphorous", "phosphorous")
plot_crop_summary(data, 'K', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of Potassium", "potassium")
plot_crop_summary(data, 'temperature', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of Temperature", "temperature")
plot_crop_summary(data, 'humidity', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of Humidity", "humidity")
plot_crop_summary(data, 'ph', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of pH", "pH")
plot_crop_summary(data, 'rainfall', 'rgba(255,0,0,0.6)', 'rgba(0,0,255,0.5)', "Summary Plot of Rainfall", "rainfall")

# Splitting Data & Normalizing

In [ ]:
# Assign features and labels
features = data[['N','P','K','temperature','humidity','ph','rainfall']].values
labels = data['label'].values

In [ ]:
# Normalize input features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

In [ ]:
# Convert label strings to numeric representations
label_to_index = {label: idx for idx, label in enumerate(np.unique(labels))}
numeric_labels = np.array([label_to_index[label] for label in labels])

In [ ]:
# Print label mappings
print("Label to Index Mapping:")
for label, index in label_to_index.items():
    print(f"{label}: {index}")

In [ ]:
# Convert labels to torch.long data type
labels = torch.tensor(numeric_labels, dtype=torch.long)

In [ ]:
# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Split the data into training and test sets while preserving label distribution
for train_index, test_index in sss.split(features_normalized, labels):
    X_train, X_test = features_normalized[train_index], features_normalized[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [ ]:
# Convert features to float32 and labels to int64
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = y_train.clone().detach()
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach()

# Neural Network Architectures

In [ ]:
# Define neural network architecture with 2 hidden layers
class NeuralNetwork2Hidden(nn.Module):
    def __init__(self, activation, n_neurons, dropout_rate, weight_init):
        super().__init__()
        self.fc1 = nn.Linear(7, n_neurons)
        self.bn1 = nn.BatchNorm1d(n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.bn2 = nn.BatchNorm1d(n_neurons)
        self.fc3 = nn.Linear(n_neurons, 22)
        self.activation = activation()
        self.dropout = nn.Dropout(dropout_rate)
        self.weight_init = weight_init
        self.initialize_weights()

    def initialize_weights(self):
        self.weight_init(self.fc1.weight)
        self.weight_init(self.fc2.weight)
        self.weight_init(self.fc3.weight)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Define neural network architecture with 5 hidden layers
class NeuralNetwork5Hidden(nn.Module):
    def __init__(self, activation, n_neurons, dropout_rate, weight_init):
        super().__init__()
        self.fc1 = nn.Linear(7, n_neurons)
        self.bn1 = nn.BatchNorm1d(n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.bn2 = nn.BatchNorm1d(n_neurons)
        self.fc3 = nn.Linear(n_neurons, n_neurons)
        self.bn3 = nn.BatchNorm1d(n_neurons)
        self.fc4 = nn.Linear(n_neurons, n_neurons)
        self.bn4 = nn.BatchNorm1d(n_neurons)
        self.fc5 = nn.Linear(n_neurons, n_neurons)
        self.bn5 = nn.BatchNorm1d(n_neurons)
        self.fc6 = nn.Linear(n_neurons, 22)
        self.activation = activation()
        self.dropout = nn.Dropout(dropout_rate)
        self.weight_init = weight_init
        self.initialize_weights()

    def initialize_weights(self):
        self.weight_init(self.fc1.weight)
        self.weight_init(self.fc2.weight)
        self.weight_init(self.fc3.weight)
        self.weight_init(self.fc4.weight)
        self.weight_init(self.fc5.weight)
        self.weight_init(self.fc6.weight)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.activation(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.activation(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = self.activation(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = self.fc6(x)
        return x

# Hyperparameter Tuning

In [ ]:
# Create models with Skorch
model_2 = NeuralNetClassifier(NeuralNetwork2Hidden)
model_5 = NeuralNetClassifier(NeuralNetwork5Hidden)

In [ ]:
# Define the parameter grid
param_grid = {
    'batch_size': [10, 50, 100],
    'max_epochs': [10, 50, 100],
    'optimizer': [optim.Adam, optim.Adadelta, optim.Adagrad, optim.Adamax],
    'optimizer__lr': [0.0001, 0.001, 0.01],
    'module__activation': [nn.ReLU, nn.LeakyReLU, nn.RReLU],
    'module__n_neurons': [10, 40, 80],
    'criterion': [nn.CrossEntropyLoss],
    'module__dropout_rate': [0.0, 0.2, 0.5],
    'module__weight_init': [init.xavier_uniform_, init.xavier_normal_]
}

In [ ]:
# Perform grid search with cross-validation to the first model
grid_2 = GridSearchCV(estimator=model_2, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=5)
grid_result_2 = grid_2.fit(X_train, y_train)

In [ ]:
# Perform grid search with cross-validation to the second model
grid_5 = GridSearchCV(estimator=model_5, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=5)
grid_result_5 = grid_5.fit(X_train, y_train)

In [ ]:
# Get the best parameters and the best score for the first model
print("Best parameters found - 2 hidden layers: ", grid_result_2.best_params_)
print("Best score found - 2 hidden layers: ", grid_result_2.best_score_)

In [ ]:
# Get the best parameters and the best score for the second model
print("Best parameters found - 5 hidden layers: ", grid_result_5.best_params_)
print("Best score found - 5 hidden layers: ", grid_result_5.best_score_)

In [ ]:
# Define data frame for results
cv_results_df_2 = pd.DataFrame(grid_result_2.cv_results_)

# Define filename with timestamp
filename_2 = f"cv_results_2_hidden_layers.xlsx"

# Save DataFrame to Excel
cv_results_df_2.to_excel(filename_2, index=False)

# Save the best model
best_model_2 = grid_result_2.best_estimator_

In [ ]:
# Define data frame for results
cv_results_df_5 = pd.DataFrame(grid_result_5.cv_results_)

# Define filename with timestamp
filename_5 = f"cv_results_5_hidden_layers.xlsx"

# Save DataFrame to Excel
cv_results_df_5.to_excel(filename_5, index=False)

# Save the best model
best_model_5 = grid_result_5.best_estimator_

# Experimentation Plots - 2 Hidden Layer Architecture

In [ ]:
# Load the cv_results from the Excel file
cv_results = pd.read_excel('cv_results_2_hidden_layers.xlsx')

# Set consistent style and color palette
sns.set_style("whitegrid")
sns.set_palette("husl")

In [ ]:
# Plot validation curve for learning rate for each optimizer
fig, axes = plt.subplots(figsize=(10, 6))
for optimizer in cv_results['param_optimizer'].unique():
    optimizer_results = cv_results[cv_results['param_optimizer'] == optimizer]
    sns.lineplot(x='param_optimizer__lr', y='mean_test_score', data=optimizer_results, label=optimizer)

In [ ]:
# Plot validation curve for batch size
fig, axes = plt.subplots(figsize=(10, 6))
sns.lineplot(x='param_batch_size', y='mean_test_score', data=cv_results, ax=axes)
axes.set_title('Validation Curve for Batch Size')
axes.set_xlabel('Batch Size')
axes.set_ylabel('Mean Test Score')
plt.show()

In [ ]:
# Heatmap for hyperparameters
pivot_table = cv_results.pivot_table(index='param_optimizer', columns='param_module__activation', values='mean_test_score')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, cmap='YlGnBu')
plt.title('Mean Test Scores for Hyperparameters')
plt.xlabel('Activation Function')
plt.ylabel('Optimizer')
plt.show()

In [ ]:
# Bar plot of mean test score for optimizers
fig, axes = plt.subplots(figsize=(10, 6))
sns.barplot(x='param_optimizer', y='mean_test_score', data=cv_results, ax=axes)
axes.set_title('Mean Test Score for Optimizers')
axes.set_xlabel('Optimizer')
axes.set_ylabel('Mean Test Score')
plt.show()

In [ ]:
# Bar plot of mean test score for activation functions
fig, axes = plt.subplots(figsize=(10, 6))
sns.barplot(x='param_module__activation', y='mean_test_score', data=cv_results, ax=axes)
axes.set_title('Mean Test Score for Activation Functions')
axes.set_xlabel('Activation Function')
axes.set_ylabel('Mean Test Score')
plt.show()

In [ ]:
# Line plot of mean test score over epochs for each combination of hyperparameters
g = sns.FacetGrid(cv_results, col='param_optimizer', hue='param_module__activation', col_wrap=3, height=4)
g.map(sns.lineplot, 'param_max_epochs', 'mean_test_score')
g.set_titles(col_template="{col_name}")
g.set_xlabels('Epochs')
g.set_ylabels('Mean Test Score')
g.add_legend(title='Activation Function')
plt.show()

In [ ]:
# Distribution plot of fit time
fig, axes = plt.subplots(figsize=(10, 6))
sns.histplot(cv_results['mean_fit_time'], kde=True, ax=axes)
axes.set_title('Distribution of Fit Time')
axes.set_xlabel('Mean Fit Time (s)')
axes.set_ylabel('Density')
plt.show()

# Experimentation Plots - 5 Hidden Layer Architecture

In [ ]:
# Load the cv_results from the Excel file
cv_results = pd.read_excel('cv_results_5_hidden_layers.xlsx')

# Set consistent style and color palette
sns.set_style("whitegrid")
sns.set_palette("husl")

In [ ]:
# Plot validation curve for learning rate for each optimizer
fig, axes = plt.subplots(figsize=(10, 6))
for optimizer in cv_results['param_optimizer'].unique():
    optimizer_results = cv_results[cv_results['param_optimizer'] == optimizer]
    sns.lineplot(x='param_optimizer__lr', y='mean_test_score', data=optimizer_results, label=optimizer)

In [ ]:
# Plot validation curve for batch size
fig, axes = plt.subplots(figsize=(10, 6))
sns.lineplot(x='param_batch_size', y='mean_test_score', data=cv_results, ax=axes)
axes.set_title('Validation Curve for Batch Size')
axes.set_xlabel('Batch Size')
axes.set_ylabel('Mean Test Score')
plt.show()

In [ ]:
# Heatmap for hyperparameters
pivot_table = cv_results.pivot_table(index='param_optimizer', columns='param_module__activation', values='mean_test_score')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, cmap='YlGnBu')
plt.title('Mean Test Scores for Hyperparameters')
plt.xlabel('Activation Function')
plt.ylabel('Optimizer')
plt.show()

In [ ]:
# Bar plot of mean test score for optimizers
fig, axes = plt.subplots(figsize=(10, 6))
sns.barplot(x='param_optimizer', y='mean_test_score', data=cv_results, ax=axes)
axes.set_title('Mean Test Score for Optimizers')
axes.set_xlabel('Optimizer')
axes.set_ylabel('Mean Test Score')
plt.show()

In [ ]:
# Bar plot of mean test score for activation functions
fig, axes = plt.subplots(figsize=(10, 6))
sns.barplot(x='param_module__activation', y='mean_test_score', data=cv_results, ax=axes)
axes.set_title('Mean Test Score for Activation Functions')
axes.set_xlabel('Activation Function')
axes.set_ylabel('Mean Test Score')
plt.show()

In [ ]:
# Line plot of mean test score over epochs for each combination of hyperparameters
g = sns.FacetGrid(cv_results, col='param_optimizer', hue='param_module__activation', col_wrap=3, height=4)
g.map(sns.lineplot, 'param_max_epochs', 'mean_test_score')
g.set_titles(col_template="{col_name}")
g.set_xlabels('Epochs')
g.set_ylabels('Mean Test Score')
g.add_legend(title='Activation Function')
plt.show()

In [ ]:
# Distribution plot of fit time
fig, axes = plt.subplots(figsize=(10, 6))
sns.histplot(cv_results['mean_fit_time'], kde=True, ax=axes)
axes.set_title('Distribution of Fit Time')
axes.set_xlabel('Mean Fit Time (s)')
axes.set_ylabel('Density')
plt.show()

# Metrics - Classification Report

In [ ]:
# Predict labels using the model with 2 hidden layers
y_pred = best_model_2.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate classification report
print("\nClassification Report for 2 hidden layers:")
print(classification_report(y_test, y_pred))

In [ ]:
# Predict labels using the model with 5 hidden layers
y_pred = best_model_5.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate classification report
print("\nClassification Report for 5 hidden layers:")
print(classification_report(y_test, y_pred))